In [1]:
!pip install textblob tqdm vaderSentiment

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from textblob import TextBlob
from itertools import combinations, chain
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#from transformers import pipeline
from tqdm import tqdm

# VADER
#analyzer = SentimentIntensityAnalyzer()

# Load a pre-trained sentiment analysis model
#model_name = "twitter-roberta-base-sentiment-latest"
#sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)



In [3]:
# List of identity characteristics
identity_characteristics = [
    'straight', 'gay', 'homosexual', 'heterosexual', 'lesbian', 'bisexual', 'transgender',
    'queer', 'non-binary', 'Black', 'African American', 'White', 'Caucasian', 'Asian', 
    'Hispanic', 'Latinx', 'Chinese', 'Indian', 'Native American',
    'terrible', 'awful', 'wonderful', 'amazing' # for calibration
    # Add more terms as needed
]

# Sample phrases with placeholders for identity characteristics
phrase_templates = [
    "I am reflecting on my experience as a {} person",
    "The next presenter will tell the story of a {} person",
    "The machine learning classifier was biased towards {} people",
    # Add more phrases as needed
]


In [4]:
# Function to compute sentiment polarity
def compute_sentiment(phrase):
    blob = TextBlob(phrase)
    score = blob.sentiment.polarity
    
    # Uncomment for VADER
    #score = analyzer.polarity_scores(text)["compound"]
    
    #print(phrase, blob.sentiment.polarity)
    return score

# Function to analyze the impact of each word using a Shapley-like approach
def analyze_impact(identity_characteristics, phrase_templates):
    impact_sums = {word: 0.0 for word in identity_characteristics}
    count_sums = {word: 0 for word in identity_characteristics}

    for phrase_template in phrase_templates:
        # Limit to combinations of max 3 identity terms
        all_subsets = list(chain(*[combinations(identity_characteristics, r) for r in range(1, 4)]))

        for subset in tqdm(all_subsets, desc="Analyzing"):
            subset_score = compute_sentiment(phrase_template.format(" ".join(subset)))
            for word in identity_characteristics:
                if word not in subset:
                    new_subset = list(subset) + [word]
                    new_subset_score = compute_sentiment(phrase_template.format(" ".join(new_subset)))
                    impact_sums[word] += new_subset_score - subset_score
                    count_sums[word] += 1

    # Calculate average impact for each word
    average_impacts = {word: impact_sums[word] / count_sums[word] for word in identity_characteristics}
    return average_impacts

In [5]:
# Analyze the impact of each word
average_impacts = analyze_impact(identity_characteristics, phrase_templates)

# Sort and display results
sorted_impacts = sorted(average_impacts.items(), key=lambda x: x[1], reverse=True)

for word, impact in tqdm(sorted_impacts, desc="Calculating Impact"):
    print(f"Word: {word}, Average Impact on Polarity: {impact}")

Calculating Impact: 100%|██████████| 23/23 [00:00<00:00, 28926.23it/s]

Word: wonderful, Average Impact on Polarity: 0.4366570614116649
Word: amazing, Average Impact on Polarity: 0.26149315238271975
Word: gay, Average Impact on Polarity: 0.18120969407779397
Word: straight, Average Impact on Polarity: 0.08632924335378352
Word: homosexual, Average Impact on Polarity: 0.0
Word: heterosexual, Average Impact on Polarity: 0.0
Word: lesbian, Average Impact on Polarity: 0.0
Word: bisexual, Average Impact on Polarity: 0.0
Word: transgender, Average Impact on Polarity: 0.0
Word: queer, Average Impact on Polarity: 0.0
Word: non-binary, Average Impact on Polarity: 0.0
Word: Caucasian, Average Impact on Polarity: 0.0
Word: Hispanic, Average Impact on Polarity: 0.0
Word: Latinx, Average Impact on Polarity: 0.0
Word: Indian, Average Impact on Polarity: 0.0
Word: White, Average Impact on Polarity: -0.001252711160686619
Word: Chinese, Average Impact on Polarity: -0.001252711160686619
Word: Native American, Average Impact on Polarity: -0.0012527111606866195
Word: Asian, Ave